In [3]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt 
import sys
import torch
import torch.optim as optim
import torch.nn as nn
from dip import EncDec
from utils import imread

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [36]:
class EncDec(nn.Module):
    def __init__(self):
        super(EncDec, self).__init__()
        self.bn16 = nn.BatchNorm2d(16)
        self.bn32 = nn.BatchNorm2d(32)
        self.bn64 = nn.BatchNorm2d(64)
        self.bn128 = nn.BatchNorm2d(128)
        self.pool1 = nn.MaxPool2d(2)
        
        self.conv1 = nn.Conv2d(1, 16, 3, 2, 1) # Encoder - 2 
        self.conv2 = nn.Conv2d(16, 32, 3, 2, 1) # Encoder - 2 
        self.conv3 = nn.Conv2d(32, 64, 3, 2, 1) # Encoder - 2 
        #self.conv4 = nn.Conv2d(64, 64, 3, 2, 1) # Encoder - 2
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear')
        #self.conv4_t = nn.Conv2d(64, 64, 3, 1, 1) # Decoder - 3
        self.conv3_t = nn.Conv2d(64, 32, 3, 1, 1) # Decoder - 3
        self.conv2_t = nn.Conv2d(32, 16, 3, 1, 1) # Decoder - 3
        self.conv1_t = nn.Conv2d(16, 1, 3, 1, 1) # Decoder - 1
        
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.bn32((F.relu(self.conv2(out))))
        out = self.bn64(F.relu(self.conv3(out)))
        #out = self.bn64(F.relu(self.conv4(out)))
        
        #out = self.bn64(F.relu(self.conv4_t(self.upsample(out))))
        out = self.bn32(F.relu(self.conv3_t(self.upsample(out))))
        out = self.bn16(F.relu(self.conv2_t(self.upsample(out))))
        out = self.conv1_t(self.upsample(out))
        
        return (out)

In [37]:
file = 'saturn'
im = imread('../data/denoising/{}.png'.format(file))
noise = imread('../data/denoising/{}-noisy.png'.format(file))
noisy_img = torch.FloatTensor(noise).unsqueeze(0).unsqueeze(0).transpose(2, 3)
clean_img = torch.FloatTensor(im).unsqueeze(0).unsqueeze(0).transpose(2,3)

MAX_EPOCHS = 5000

#for i in range(10):
net = EncDec()
criterion, optimizer = nn.MSELoss(), optim.Adam(net.parameters(), lr=0.01)
eta = torch.randn(*noisy_img.size())

for itr in range(MAX_EPOCHS+1):
    out = net(eta)
    optimizer.zero_grad()
    loss = criterion(out, noisy_img)
    loss.backward()
    optimizer.step()

    if itr %100 == 0:
        op_img = net(eta)[0, 0, :, :].transpose(0,1).detach().numpy()
        error = ((op_img - noise)**2).sum()
        print ("Loss = %.4f" % error)
print ('----------')

Loss = 24183.7833
Loss = 470.9724
Loss = 408.9066
Loss = 385.0811
Loss = 357.5202
Loss = 341.0163


KeyboardInterrupt: 